# 语义分割

我们已经学习了如何识别图片里面的主要物体，和找出里面物体的边框。语义分割则在纸上更进一步，它对每个像素预测它是否是背景，还是属于哪个我们感兴趣的物体。

跟物体检测相比，语义分割预测的边框更加精细。

与图片分割的异同：它跟语义分割类似，将每个像素划分的某个类，但它跟语义分割不同的是，图片分割不需要预测每个类具体对应哪个物体。因此图片分割经常只需要利用像素之间的相似度即可，而语义分割则需要更详细的类别标号，这也是为什么称为语义的原因。

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import image 
from mxnet import autograd

## VOC2012数据集

VOC2012是一个常用的语义分割数据集。输入图片跟之前的数据集类似，但标注也是保存成相应大小的图片来方便查看，下面代码下载这个数据集并解压。

In [ ]:
import os
import tarfile

data_root = '../data'
voc_root = data_root + '/VOCdevkit/VOC2012/'
url = ('http://host.robots.ox.ac.uk/pascal/VOC/voc2012'
       '/VOCtrainval_11-May-2012.tar')
sha1 = '4e443f8a2eca6b1dac8a6c57641b67dd40621a49'

fname = gluon.utils.download(url, data_root, sha1_hash=sha1)

if not os.path.isfile(voc_root + '/ImageSets/Segmentation/train.txt'):
    with tarfile.open(fname, 'r') as f:  
        f.extractall(data_root)

下面定义函数将训练图片和标注按序读进内存

In [5]:
def read_images(root=voc_root, train=True):
    txt_fname = root + 'ImageSets/Segmentation/' + ('train.txt' if train else 'val.txt')
    with open(txt_fname, 'r') as f:
        images = f.read().split()
    n = len(images)
    data, label = [None] * n, [None] * n
    for i, fname in enumerate(images):
        data[i] = image.imread('%s/JPEGImages/%s.jpg' % (root, fname))
        label[i] = image.imread('%s/SegmentationClass/%s.png' % (root, fname))
    return data, label

我们画出前面三张图片和他们对应的标号。在标号中，白色代表边框黑色代表背景，其他不同的颜色对应不同物体

In [6]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt

In [8]:
def show_images(imgs, nrows, ncols, figsize=None):
    if not figsize:
        figsize = (nrows, ncols)
    _, figs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    for i in range(nrows):
        for j in range(ncols):
            figs[i][j].imshow(imgs[i*ncols+j].asnumpy())
            figs[i][j].axes.get_xaxis().set_visible(False)
            figs[i][j].axes.get_yaxis().set_visible(False)
    plt.show()

In [11]:
train_images, train_labels = read_images()

imgs = []
for i in range(3):
    imgs += [train_images[i], train_labels[i]]
    
show_images(imgs, nrows=3, ncols=2, figsize=(12, 8))

FileNotFoundError: [Errno 2] No such file or directory: '../data/VOCdevkit/VOC2012ImageSets/Segmentation/train.txt'

In [10]:
[im.shape for im in imgs]

NameError: name 'imgs' is not defined